# Detecting Faces

## Imports

In [25]:
import pandas as pd
import time

## Data Visialisation & Structuring

In [26]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
from collections import Counter

In [27]:
#from torchsampler import ImbalancedDatasetSampler

In [28]:
# Transfer data to tensor vectors and training/ testing datasets
train_dir = './start_deep/train_images'
test_dir = './start_deep/test_images'

transform = transforms.Compose(
    [transforms.Grayscale(), 
     transforms.ToTensor(), 
     transforms.Normalize(mean=(0,),std=(1,))])

train_data = torchvision.datasets.ImageFolder(train_dir, transform=transform)
test_data = torchvision.datasets.ImageFolder(test_dir, transform=transform)

valid_size = 0.2
batch_size = 32

num_train = len(train_data)
print("len(train_data)",len(train_data))
print ("len(test_data)", len(test_data))
indices_train = list(range(num_train))
np.random.shuffle(indices_train)
split_tv = int(np.floor(valid_size * num_train))
train_new_idx, valid_idx = indices_train[split_tv:],indices_train[:split_tv]

train_sampler = SubsetRandomSampler(train_new_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=train_sampler, num_workers=1)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler, num_workers=1)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=1)
classes = ('noface','face')

# for epoch in range(1, n_epochs+1):
# for data, target in train_loader:


len(train_data) 91720
len(test_data) 7628


In [29]:
print(dict(Counter(train_data.targets)))

{0: 26950, 1: 64770}


In [30]:
print(dict(Counter(test_data.targets)))

{0: 6831, 1: 797}


In [31]:
count_test_face = 0
count_test_no_face = 0
for data, target in test_loader:
    s = np.array(target).sum()
    count_test_face +=s
    count_test_no_face += batch_size - s

In [49]:
print(count_test_face, count_test_no_face)

797 6851


In [45]:
count_train_face = 0
count_train_no_face = 0
for data, target in train_loader:
    s = np.array(target).sum()
    count_train_face +=s
    count_train_no_face += batch_size - s

In [46]:
print(count_train_face, count_train_no_face)

51778 21598


In [47]:
count_valid_face = 0
count_valid_no_face = 0
for data, target in valid_loader:
    s = np.array(target).sum()
    count_valid_face +=s
    count_valid_no_face += batch_size - s

In [48]:
print(count_valid_face, count_valid_no_face)

12992 5376


## Building Model

In [40]:
###############################################
# Confusion Matrix 
# Input : output (of model), labels
# Output: average error
###############################################
class ConfusionMatrixMeter(object):     #same as for repSet
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.true_pos = 0
        self.true_neg = 0
        self.false_pos = 0
        self.false_neg = 0

    def update(self, output, target):
        for i in range(len(output)):
            if output[i][0]>output[i][1]: # predicted negative (no face)
                if target[i]==0:
                    self.true_neg +=1
                else:
                    self.false_neg +=1
            else:                         # predicted positive (face)
                if target[i]==1:
                    self.true_pos +=1
                else:
                    self.false_pos +=1



In [41]:
###############################################
# Description: error function to evaluate distance between prediction and label 
# Input : output (of model), labels
# Output: average error
###############################################
def error(output, labels):
    #print ("output",output)
    #print ("labels",labels)
    
    correct = 0
    predicted = torch.max(output.data, 1)[1] 
    correct += (predicted == labels).sum()
    return 1 - correct.item()/len(output.data)

In [9]:
###############################################
# Description: class used to keep count of the average error and loss during training
# Input : 
# Output:
###############################################
class AverageMeter(object):     #same as for repSet
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import log_loss

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Model 1: default
class ConvNet(nn.Module):
    
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(9 * 9 * 64, 100)
        self.fc2 = nn.Linear(100, 2)
        #self.sigmoid = nn.Sigmoid()
    
    def forward(self, X): # batch of n images
        #print ("input",X.size())
        out = self.layer1(X)
        #print ("after layer1",out.size())
        out = self.layer2(out)
        #print ("after layer2",out.size())
        out = out.reshape(out.size(0), -1)
        #print ("after reshape",out.size())
        out = self.drop_out(out)
        #print ("after drop",out.size())
        out = self.fc1(out)
        #print ("after fc1",out.size())
        out = self.fc2(out)
        #print ("after fc2",out.size())
        #out = F.log_softmax(out, dim=1)
        #print ("after log softmax",out.size())
        return out 

## Evaluating Model

In [42]:
# Hyperparameters
epochs = 3
lr = 0.001
cv_folds = 1

In [44]:
for it in range(cv_folds):
    
    ### Set X and y tensors for training set ###
    n_train_batches =200
    n_valid_batches = 50
    ### Set X and y tensors for testing set ###
    n_test_batches = 250
        ############################################ - TRAIN, TEST FUNCTIONS
    def train(X, y):
        optimizer.zero_grad()
        output = model(X)
        loss_train = F.cross_entropy(output, y)
        #print (output)
        #print (y)
        loss_train.backward()
        optimizer.step()
        return output, loss_train
    
    def test(X, y):
        output = model(X)
        loss_test = F.cross_entropy(output, y)
        return output, loss_test 

    #Initialize Model
    model = ConvNet().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model.train()
    
    start = time.perf_counter()       #time indicator 
    for epoch in range(epochs):
        
        model.train()
        startEpoch = time.perf_counter()    #time indicator
        train_loss = AverageMeter()
        train_err = AverageMeter()
        i =0
        for data, target in train_loader:
            i+=1
            if i == n_train_batches:
                break
            output, loss = train(data, target)
            train_loss.update(loss.item(), output.size(0))
            train_err.update(error(output, target), output.size(0))
            
        model.eval()
        test_loss = AverageMeter()
        test_err = AverageMeter()
        i = 0
        for data, target in valid_loader:
            i+=1
            if i == n_valid_batches:
                break
            output, loss = test(data, target)
            test_loss.update(loss.item(), output.size(0))
            test_err.update(error(output, target), output.size(0)) 
        print("Cross-val iter:", '%02d' % (it+1), "epoch:", '%03d' % (epoch+1), "train_loss=", "{:.5f}".format(train_loss.avg),
        "train_err=", "{:.5f}".format(train_err.avg), "val_loss=", "{:.5f}".format(test_loss.avg),
        "val_err=", "{:.5f}".format(test_err.avg), "epoch_time=", "{:.5f}".format(time.perf_counter()-startEpoch))
    
    model.eval()
    test_loss = AverageMeter()
    test_err = AverageMeter()
    i=0
    cmm = ConfusionMatrixMeter()
    for data, target in test_loader:
        i+=1
        if i == n_test_batches:
            break
        output, loss = test(data, target)
        cmm.update(np.array(output.detach()), np.array(target))
        test_loss.update(loss.item(), output.size(0))
        test_err.update(error(output.data, target.data), output.size(0)) 
    print("Cross-val iter:", '%02d' % (it+1),"test_loss=", "{:.5f}".format(test_loss.avg),
        "test_err=", "{:.5f}".format(test_err.avg), "cv time=", "{:.5f}".format(time.perf_counter() -start))
    print (cmm.true_pos, cmm.false_pos)
    print (cmm.false_neg, cmm.true_neg)

Cross-val iter: 01 epoch: 001 train_loss= 0.27163 train_err= 0.11354 val_loss= 0.09809 val_err= 0.03890 epoch_time= 38.45513
Cross-val iter: 01 epoch: 002 train_loss= 0.11648 train_err= 0.04287 val_loss= 0.05773 val_err= 0.02105 epoch_time= 32.87979
Cross-val iter: 01 epoch: 003 train_loss= 0.07605 train_err= 0.02356 val_loss= 0.04219 val_err= 0.01658 epoch_time= 33.94685
Cross-val iter: 01 test_loss= 0.15697 test_err= 0.04588 cv time= 117.90147
517 70
280 6761
